# Implementing the online phase, assuming a trusted dealer
We want to make secure computation on Secre-shared values, this phase takes secret inputs, performs arithmetic, and uses Breaver triples to ensure nobody cheats. 
Before revealing any output, the protocol checks α·a* == sum(gamma*_i)

Finally, the protocol reveals the result with a global MAC check. 

In [2]:
from dataclasses import dataclass
from typing import List
import random

In [ ]:
# Large prime modulus (field $F_p$)
P = 2**61 - 1

def mod(x: int) -> int:
    return x % P

@dataclass
class ShareRepr:
    delta: int
    shares: List[int]
    mac_shares: List[int]

class TrustedDealer:
    def __init__(self, n: int):
        self.n = n
        # Generate global MAC key alpha
        self.alpha = mod(random.randrange(P))

        # Secret-share alpha: alpha = sum(alpha_i)
        self.alpha_shares = [mod(random.randrange(P)) for _ in range(n-1)]
        last = mod(self.alpha - sum(self.alpha_shares))
        self.alpha_shares.append(last)

        self.triples = []
        self.pairs = []

    def make_secret_shared(self, v: int):
        # additive shares
        shares = [mod(random.randrange(P)) for _ in range(self.n-1)]
        shares.append(mod(v - sum(shares)))

        # MAC shares: gamma_i = alpha_i * (v + delta)
        delta = 0
        mac_shares = [mod(self.alpha_shares[i] * (v + delta)) for i in range(self.n)]

        return shares, mac_shares

    def new_pair(self):
        r = mod(random.randrange(P))
        shares, mac_shares = self.make_secret_shared(r)
        r_repr = ShareRepr(delta=0, shares=shares, mac_shares=mac_shares)
        self.pairs.append((r_repr, r))
        return r_repr, r

    def new_triple(self):
        a = mod(random.randrange(P))
        b = mod(random.randrange(P))
        c = mod(a * b)

        shares_a, mac_a = self.make_secret_shared(a)
        shares_b, mac_b = self.make_secret_shared(b)
        shares_c, mac_c = self.make_secret_shared(c)

        a_repr = ShareRepr(0, shares_a, mac_a)
        b_repr = ShareRepr(0, shares_b, mac_b)
        c_repr = ShareRepr(0, shares_c, mac_c)

        self.triples.append((a_repr, b_repr, c_repr))
        return a_repr, b_repr, c_repr


class OnlineProtocol:
    def __init__(self, n: int, dealer: TrustedDealer):
        self.n = n
        self.dealer = dealer
        self.reprs = {}  # label → ShareRepr
        self.opened_values = []  # for final MAC check

    # ------------------------
    # INPUT PROTOCOL
    def input_share(self, owner: int, value: int, label: str):
        r_repr, r = self.dealer.pairs.pop(0) if self.dealer.pairs else self.dealer.new_pair()

        eps = mod(value - r)  # owner broadcasts

        shares = r_repr.shares.copy()
        macs = r_repr.mac_shares.copy()

        # Add eps to share of player 0 (equivalent, but any position works)
        shares[0] = mod(shares[0] + eps)
        macs[0] = mod(macs[0] + mod(self.dealer.alpha_shares[0] * eps))

        x_repr = ShareRepr(delta=0, shares=shares, mac_shares=macs)
        self.reprs[label] = x_repr
        return x_repr

    # ------------------------
    # ADDITION
    # ------------------------
    def add(self, lx: str, ly: str, lout: str):
        X = self.reprs[lx]
        Y = self.reprs[ly]

        shares = [mod(X.shares[i] + Y.shares[i]) for i in range(self.n)]
        macs = [mod(X.mac_shares[i] + Y.mac_shares[i]) for i in range(self.n)]

        self.reprs[lout] = ShareRepr(delta=0, shares=shares, mac_shares=macs)
        return self.reprs[lout]

    # ------------------------
    # MULTIPLICATION (Beaver)
    # ------------------------
    def multiply(self, lx: str, ly: str, lout: str):
        a_repr, b_repr, c_repr = (
            self.dealer.triples.pop(0) if self.dealer.triples else self.dealer.new_triple()
        )

        X = self.reprs[lx]
        Y = self.reprs[ly]

        x = mod(sum(X.shares))
        y = mod(sum(Y.shares))
        a = mod(sum(a_repr.shares))
        b = mod(sum(b_repr.shares))

        eps = mod(x - a)
        delt = mod(y - b)

        # record as opened values for MAC check
        self._record_open(eps)
        self._record_open(delt)

        # compute z
        c = mod(sum(c_repr.shares))
        z = mod(c + eps*b + delt*a + eps*delt)

        # new shares
        shares = [mod(random.randrange(P)) for _ in range(self.n-1)]
        shares.append(mod(z - sum(shares)))

        macs = [mod(self.dealer.alpha_shares[i] * z) for i in range(self.n)]
        z_repr = ShareRepr(delta=0, shares=shares, mac_shares=macs)
        self.reprs[lout] = z_repr
        return z_repr

    def _record_open(self, val: int):
        mac = [mod(self.dealer.alpha_shares[i] * val) for i in range(self.n)]
        self.opened_values.append((val, 0, mac))

    # ------------------------
    # OUTPUT + MAC CHECK
    # ------------------------
    def output_open(self, label: str):
        Y = self.reprs[label]
        T = len(self.opened_values)

        if T > 0:
            e = [mod(random.randrange(P)) for _ in range(T)]
            a_star = 0
            mac_star = [0]*self.n
            for j,(v,delta,macs) in enumerate(self.opened_values):
                a_star = mod(a_star + e[j]*v)
                for i in range(self.n):
                    mac_star[i] = mod(mac_star[i] + e[j]*macs[i])

            left = mod(self.dealer.alpha * a_star)
            right = mod(sum(mac_star))
            if left != right:
                raise Exception("MAC linear combination check FAILED.")

        y = mod(sum(Y.shares))
        if mod(self.dealer.alpha * y) != mod(sum(Y.mac_shares)):
            raise Exception("Final output MAC check FAILED.")

        return y


In [ ]:
# 7 * 11 using online phase
# --------------------------
def demo():
    random.seed(1)

    n = 3
    dealer = TrustedDealer(n)

    # Preprocessing: generate enough masks/triples
    for _ in range(5):
        dealer.new_pair()
    for _ in range(5):
        dealer.new_triple()

    # Online phase
    proto = OnlineProtocol(n, dealer)

    # Player 0 inputs x = 7
    proto.input_share(owner=0, value=7, label="x")

    # Player 1 inputs y = 11
    proto.input_share(owner=1, value=11, label="y")

    # Multiply z = x * y
    proto.multiply("x", "y", "z")

    # Open result with full MAC check
    result = proto.output_open("z")

    print("Computed result:", result)

demo()


Computed result: 77
